## **임대주택 단지 내 적정 주차수요 예측**

#### 주제 
유형별 임대주택 설계 시 단지 내 적정 주차 수요 예측


#### **평가**  
- MAE(Mean Absolute Error)
- public : 전체 TEST 데이터 중 무작위 33%
- private : 전체 TEST 데이터 중 무작위 67%

#### 외부 데이터 및 사전 학습 모델
- 공공 데이터와 같이 누구나 얻을 수 있고 법적 제약이 없는 외부 데이터 허용
- 사전학습 모델의 경우 사전학습에 사용된 데이터를 명시
- 최종 평가시 외부데이터 및 출처 제출

#### 데이터 오류 처리 방안
 
-  테스트셋에서 평가 제외되는 데이터는 'C2675'(2번 사항에 해당), 'C2335', 'C1327'(3번 사항에 해당) 3개 단지  
   ['C2675','C2335', 'C1327']
-  Train 제거  
   ['C2085', 'C1397', 'C2431', 'C1649', 'C1036', 'C2675', 'C2335', 'C1327', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']






In [234]:
import pandas
import numpy
import os
import math
import warnings
from IPython.core.display import display, HTML

## 전체 구간을 넓게
display(HTML("<style>.container { width:100% !important; }</style>"))
## 각 컬럼 width 최대로
pandas.set_option('display.max_colwidth', -1)
## rows 500
pandas.set_option('display.max_rows', 500)
## columns
pandas.set_option('display.max_columns', 500)
pandas.set_option('display.width', 1000)

warnings.filterwarnings(action = 'ignore')

PATH = 'C:/Users/WAI/OneDrive/04.DOMHWANGCHA/DOMHWANGCHA/99.DATA/98.DACON/01.주차수요예측AI경진대회/'

In [183]:
train_df = pandas.read_csv(PATH + "train.csv", index_col = False)
test_df = pandas.read_csv(PATH + "test.csv", index_col = False)
df = train_df.append(test_df)
age_df = pandas.read_csv(PATH + "age_gender_info.csv", index_col=False)

print("Training Set Shape : {}".format(train_df.shape))
print("Test Set Shape : {}".format(test_df.shape))
print("Total Set Shape : {}".format(df.shape))
print("Age Gender Set Shape : {}".format(age_df.shape))

Training Set Shape : (2952, 15)
Test Set Shape : (1022, 14)
Total Set Shape : (3974, 15)
Age Gender Set Shape : (16, 23)


In [19]:
# sweetviz 라이브러리를 사용하여 EDA 결과 html로 출력
import sweetviz

eda_report = sweetviz.compare(train_df, test_df, "등록차량수")
eda_report.show_html()


Done! Use 'show' commands to display/save.   |██████████| [100%]   00:01 -> (00:00 left)


Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [248]:

train_df.loc[train_df["단지코드"] == "C1350"]["전용면적별세대수"].sum()
train_df.loc[train_df["단지코드"] == "C1350"]
# 분양의 경우 임대료 X
# 

len(train_df["단지코드"].unique())

423

In [184]:
# Train Set EDA

train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
총세대수,2952.0,886.661247,513.540168,26.00,513.50,779.00,1106.0000,2568.0
전용면적,2952.0,44.757215,31.874280,12.62,32.10,39.93,51.5625,583.4
전용면적별세대수,2952.0,102.747967,132.640159,1.00,14.00,60.00,144.0000,1865.0
공가수,2952.0,12.921070,10.778831,0.00,4.00,11.00,20.0000,55.0
도보 10분거리 내 지하철역 수(환승노선 수 반영),2741.0,0.176578,0.427408,0.00,0.00,0.00,0.0000,3.0
도보 10분거리 내 버스정류장 수,2948.0,3.695726,2.644665,0.00,2.00,3.00,4.0000,20.0
단지내주차면수,2952.0,601.668360,396.407072,13.00,279.25,517.00,823.0000,1798.0
등록차량수,2952.0,559.768293,433.375027,13.00,220.00,487.00,770.0000,2550.0


In [213]:
# Null 값 확인
# 임대보증금, 임대료 Null 존재 
train_df.isnull().sum().to_frame().T



,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,0,0,0,0,0,0,0,0,0,581,590,211,4,0,0


In [243]:
# 임대보증금과 임대료가 Null 또는 비어있는 값으로 들어가 있는 경우 0 처리
# "-" 문자열 Null로 치환 

train_df = train_df.replace("-", numpy.nan)

# 1. 임대보증금, 임대료가 NaN인 데이터에서 상가 아파트에 따른 정보 확인 
print("임대보증금이 Null 데이터 : \n {}".format(train_df[train_df["임대보증금"].isnull() == True].groupby(["임대건물구분"]).count()["단지코드"]))
print("---------------------------------------------------------")
# 상가 : 562건, 아파트: 19 건
train_df_apt = train_df[train_df["임대보증금"].isnull() == True].loc[train_df["임대건물구분"] == "아파트"]
train_df_store = train_df[train_df["임대보증금"].isnull() == True].loc[train_df["임대건물구분"] == "상가"]

train_df_apt

임대보증금이 Null 데이터 : 
 임대건물구분
상가     562
아파트    19 
Name: 단지코드, dtype: int64
---------------------------------------------------------


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2331,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2332,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2333,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2334,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2335,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2336,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2337,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2547,C1326,1934,아파트,부산광역시,국민임대,24.72,472,43.0,H,NaN,NaN,0.0,4.0,1670.0,1153.0
2548,C1326,1934,아파트,부산광역시,국민임대,24.79,104,43.0,H,NaN,NaN,0.0,4.0,1670.0,1153.0
2549,C1326,1934,아파트,부산광역시,국민임대,26.83,590,43.0,H,NaN,NaN,0.0,4.0,1670.0,1153.0


In [ ]:
# 임대료가 Null인경우 
print("임대료가 Null 데이터 : \n {}".format(train_df[train_df["임대료"].isnull() == True].groupby(["임대건물구분"]).count()["단지코드"]))
# 상가 : 562건 , 아파트 : 62건

In [ ]:
# 3. 지하철역수 가 nan인 데이터에서 지역 정보에 따른 데이터확인 여하여 Null 보정
print("---------------------------------------------------------")
print("지하철역 Null 데이터 : \n {}".format(train_df[train_df["도보 10분거리 내 지하철역 수(환승노선 수 반영)"].isnull() == True].groupby(["임대건물구분"]).count()["단지코드"]))
# 경남 : 4건 , 대전 : 93건 , 충청남도: 114건 
# 부산은 따로 표현되므로 지하철의 경우 경남, 충청남도는 지하철 노선이 존재하지 않으므로 0값 으로 채울 수 있음
# 경남, 충청은 Null값 0을 보정 가능
# 대전 Case 세부분할 확인

In [237]:
# 4. 버스가 Nan 인 데이터 지역정보 확인
print("---------------------------------------------------------")
print("버스정류장 Null 데이터 : \n {}".format(train_df[train_df["도보 10분거리 내 버스정류장 수"].isnull() == True].groupby(["지역"]).count()["단지코드"]))
# 경상남도 : 4건 



# 버스정류장의 경우 신축아파트의 경우 노선이 확정되지 않아 없는 경우가 존재할 수 있으나, 
# 데이터 확인결과 공공임대 아파트인 점을 보았을때, 표기오류로 판단할 수 있다. 
# 0으로 값을 채워넣기 보단 공공임대 아파트의 평균 정류장수를 사용하여 결측치를 보정할 수 있을 것같다.
print(train_df[train_df["도보 10분거리 내 버스정류장 수"].isnull() == True]) 

# 총 251건의 공공임대 아파트 유형 존재
print(len(train_df[train_df["공급유형"].str.contains("공공임대")]))

print("공공임대 아파트의 평균 버스정류장 수 : {} 개".format(math.floor(train_df[train_df["공급유형"].str.contains("공공임대")]["도보 10분거리 내 버스정류장 수"].dropna().mean())))
# 평균 3개의 버스정류장이 존재 
# 버스정류장의 Null 값은 3으로 보정하여 수행

---------------------------------------------------------
버스정류장 Null 데이터 : 
 지역
경상남도    4
Name: 단지코드, dtype: int64
       단지코드  총세대수 임대건물구분    지역       공급유형   전용면적  전용면적별세대수   공가수 자격유형     임대보증금     임대료  도보 10분거리 내 지하철역 수(환승노선 수 반영)  도보 10분거리 내 버스정류장 수  단지내주차면수  등록차량수
2315  C1649  575   아파트    경상남도  공공임대(10년)  74.97  80        15.0  A    46000000  456000 NaN                           NaN                  1066.0   855.0
2316  C1649  575   아파트    경상남도  공공임대(10년)  84.95  124       15.0  A    57000000  462000 NaN                           NaN                  1066.0   855.0
2317  C1649  575   아파트    경상남도  공공임대(10년)  84.96  289       15.0  A    57000000  462000 NaN                           NaN                  1066.0   855.0
2318  C1649  575   아파트    경상남도  공공임대(10년)  84.98  82        15.0  A    57000000  462000 NaN                           NaN                  1066.0   855.0
251
공공임대 아파트의 평균 버스정류장 수 : 3 개


In [180]:
# 임대료 보정하기 전에 Null 값의 유형을 좀 파악 해보자!

train_df["임대보증금"] = train_df["임대보증금"].fillna("0")
train_df["임대료"] = train_df["임대료"].fillna("0")

# 지하철역수 및 버정류장수 Null 보정

train_df["도보 10분거리 내 버스정류장 수"] = train_df["도보 10분거리 내 버스정류장 수"].fillna(0)
train_df["도보 10분거리 내 버스정류장 수"] = train_df["도보 10분거리 내 버스정류장 수"].fillna(0)

In [181]:
# 임대 보증금과 임대료 Object type -> 수치형으로 변환
train_df = train_df.astype({"임대보증금" : int, "임대료" : int}) 

train_df.describe().T


,count,mean,std,min,25%,50%,75%,max
총세대수,2952.0,8.866612e+02,5.135402e+02,26.00,513.50,779.00,1.106000e+03,2568.0
전용면적,2952.0,4.475722e+01,3.187428e+01,12.62,32.10,39.93,5.156250e+01,583.4
전용면적별세대수,2952.0,1.027480e+02,1.326402e+02,1.00,14.00,60.00,1.440000e+02,1865.0
공가수,2952.0,1.292107e+01,1.077883e+01,0.00,4.00,11.00,2.000000e+01,55.0
임대보증금,2952.0,2.157700e+07,2.212032e+07,0.00,8731000.00,17477000.00,2.820275e+07,216423000.0
임대료,2952.0,1.566010e+05,1.404528e+05,0.00,75520.00,135960.00,2.100350e+05,1058030.0
도보 10분거리 내 지하철역 수(환승노선 수 반영),2741.0,1.765779e-01,4.274079e-01,0.00,0.00,0.00,0.000000e+00,3.0
도보 10분거리 내 버스정류장 수,2952.0,3.690718e+00,2.646368e+00,0.00,2.00,3.00,4.000000e+00,20.0
단지내주차면수,2952.0,6.016684e+02,3.964071e+02,13.00,279.25,517.00,8.230000e+02,1798.0
등록차량수,2952.0,5.597683e+02,4.333750e+02,13.00,220.00,487.00,7.700000e+02,2550.0
